In [7]:
# all the imports to be able to follow along with https://www.kaggle.com/thebrownviking20/topic-modelling-with-spacy-and-scikit-learn

In [1]:
# Usual imports
import numpy as np
import pandas as pd
from tqdm import tqdm
import string
import matplotlib.pyplot as plt
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.manifold import TSNE
import concurrent.futures
import time
import pyLDAvis.sklearn
from pylab import bone, pcolor, colorbar, plot, show, rcParams, savefig
import warnings
warnings.filterwarnings('ignore')

/Users/ianm/anaconda/lib/python3.6/site-packages/past/translation/__init__.py:35: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp


In [2]:
%matplotlib inline
import os

In [3]:
# Plotly based imports for visualization
from plotly import tools
import plotly.plotly as py
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.figure_factory as ff

In [4]:
# spaCy based imports
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

In [5]:
import spacy

In [6]:
nlp = spacy.load("en_core_sci_sm")

In [8]:
# all the imports to be able to follow along with https://www.analyticsvidhya.com/blog/2018/04/a-comprehensive-guide-to-understand-and-implement-text-classification-in-python/

In [7]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

Using TensorFlow backend.


# OK, we have the machinery, let's start looking at the data

In [8]:
%store  -r 

In [9]:
# let's get this into a data frame with classifications

In [9]:
labels = []
abstracts = [] 
for abstract in unique_selected_abstracts:
    abstracts.append(abstract)
    labels.append("selected")
for abstract in rejected_abstracts:
    abstracts.append(abstract)
    labels.append("rejected")

In [10]:
trainDF = pandas.DataFrame()

In [11]:
trainDF['text'] = abstracts
trainDF['label'] = labels

In [12]:
trainDF[0:3]

,text,label
0,,selected
1,Introduction: Orthodontic treatment is reimbur...,selected
2,BackgroundThis study examined whether environm...,selected


In [13]:
len(trainDF)

1071

In [14]:
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(trainDF['text'], trainDF['label'])

In [15]:
trainDF['text'][3]

"In precision medicine, a patient is treated with targeted therapies that are predicted to be effective based on the patient's baseline characteristics such as biomarker profiles. Oftentimes, patient subgroups are unknown and must be learned through inference using observed data. We present SCUBA, a Subgroup ClUster-based Bayesian Adaptive design aiming to fulfill two simultaneous goals in a clinical trial, 1) to treatments enrich the allocation of each subgroup of patients to their precision and desirable treatments and 2) to report multiple subgroup-treatment pairs (STPs). Using random partitions and semiparametric Bayesian models, SCUBA provides coherent and probabilistic assessment of potential patient subgroups and their associated targeted therapies. Each STP can then be used for future confirmatory studies for regulatory approval. Through extensive simulation studies, we present an application of SCUBA to an innovative clinical trial in gastroesphogeal cancer."

# Try some named entity recognition

In [16]:
doc = nlp(trainDF['text'][7])

In [17]:
spacy.displacy.render(doc, style='ent',jupyter=True)